# Support Vector Machines

Support Vector Machine is a versatile algorithm that can be applied to linear or non-linear datasets. The idea behind support vector machines is to create a decision boundary between classes such that the margins between both classes are maximized. 

If instances are to be strictly imposed off the margins, this is called hard margin classification. However this method is sensitive to outliers and only works if the data is linearly separable. 

Alternatively, soft margin classification can be employed for more flexibility. The idea is to optimize the trade-off between margin violations and keeping the margins large. Allowing some observations to fall on the wrong side of the margin makes the margin 'soft'.
 
# Kernels

The SVM allows us to utilize different transformative kernels which can be used depending on the nature of our dataset. These include linear, polynomial and RBF kernel (among others). The model works best on small to medium scale datasets with high complexity.

**Polynomial Kernel**

For non-linear datasets, sometimes applying quadratic or polynomial transformation can result in a linearly separable dataset. This then allows us to apply linear SVM. However, low polynomial degrees won’t be effective with very complex datasets while high polynomials tend to create numerous features, which can cause a combinatorial explosion thereby reducing model efficiency. 

SVM's apply kernel tricks that allow us to achieve the results produced by polynomial transformations without actually increasing the number of features. 

**Gaussian Radial Basis Function Kernel**

This kernel is particularly useful for overlapping features. The Radial kernel utilizes a similarity metric to establish closely related observations. The smaller the distance the greater the using influence these observations have on each other. The gamma hyperparameter is tweaked to perform regularization.



# SVM Hyperparameters

The C hyperparameter can be tweaked to control the balance between margins and margin violations. Lower values of C leads to wider margins but more margin violations. To the contrary, using a high C value produces a narrow margin and fewer margin violation. Lower values of C increase bias (regularization) while Higher values of C increase variance.


# SVM Pros and Cons


**Pros**

- Highly flexible, particularly due to kernel tricks.

**Cons**

 - Support vector machines expect features to be scaled.
 
 - Inefficient to scale on large datasets.
 
 - Do not provide probability estimates.




# 1. Libraries

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# Import Data
df = pd.read_csv('LungCapData.csv')
df.head()

,LungCap,Age,Height,Smoke,Gender,Caesarean
0,6.475,6,62.1,no,male,no
1,10.125,18,74.7,yes,female,no
2,9.550,16,69.7,no,female,yes
3,11.125,14,71.0,no,male,no
4,4.800,5,56.9,no,male,no


# 2. Preprocessing

In [3]:
# Predictors and Target
X = df.drop(columns = ['LungCap'])
y = df['LungCap']

# Instantiate one-hot encoder
ohe = OneHotEncoder()

# columns to be one hot encoded
ct = make_column_transformer(

    (ohe, ['Smoke', 'Gender', 'Caesarean']),
    remainder = 'passthrough')

# predictors and target variable
X = np.array(ct.fit_transform(X))
y = np.array(y)

# Checck input and target variable shape
X.shape, y.shape

((725, 8), (725,))

In [4]:
# Training and Testing subsets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 911)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print('Standardized feature Mean:',  X_train.mean().round())
print('Standardized feature SD :',   X_train.std().round())

Standardized feature Mean: 0.0
Standardized feature SD : 1.0


# 3. Training

In [5]:
# Training the SVM Classifier
svr = SVR(kernel = 'linear')
svr.fit(X_train, y_train)

SVR(kernel='linear')

# 4. Testing


In [6]:
# Predicting the Test set results
y_pred = svr.predict(X_test)

# Mean squared error
print('Mean Squared Error :', mean_squared_error(y_test, y_pred))

Mean Squared Error : 1.0039382581330274


# 5. K-Fold Cross Validation

In [7]:
# 10 fold cross validation
R2 = cross_val_score(estimator = svr,
                             X = X,
                             y = y,
                             cv = 10)

# Cross validation accuracy and standard deviation
print(R2)
print("R2: {:.3f} %".format(R2.mean()*100))
print("R2 Standard Deviation: {:.3f} %".format(R2.std()*100))

[0.82962924 0.83983211 0.88385988 0.7823923  0.85396122 0.88573589
 0.84117599 0.8524749  0.87046444 0.79728231]
R2: 84.368 %
R2 Standard Deviation: 3.227 %


# 6. Hyperparametric Tuning

In [8]:
# Grid Search CV
parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
              ]


# Configure GridSearchCV
grid_search = GridSearchCV(estimator = svr,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = 10,
                           n_jobs = -1)

# Initiate Search
grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Print Results
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Best Accuracy: 84.17 %
Best Parameters: {'C': 1, 'kernel': 'linear'}


In [9]:
# Randomized Search CV
parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},]


# Configure GridSearchCV
random_search = RandomizedSearchCV(estimator = svr,
                           param_distributions = parameters,
                           scoring = 'r2',
                           cv = 10,
                           n_jobs = -1)

# Initiate Search
random_search.fit(X_train, y_train)

# Extract Tuned Parameters and Predictive Accuracy
tuned_params = random_search.best_params_
tuned_score = random_search.best_score_
best_estimator = random_search.best_estimator_

# Print Results
print("Best Accuracy: {:.2f} %".format(random_search.best_score_*100))
print("Best Parameters:", tuned_params)

Best Accuracy: 84.17 %
Best Parameters: {'kernel': 'linear', 'C': 1}
